# Notes - Implementation CNN

In [1]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
tf.__version__

'2.3.1'

## Part 1 - Data Preprocessing
### Preprocessing the Training set

* We will apply some Transformations on the Images in the Training set only, thats because to avoid the Overfitting.
* Some Geometrical transformations in the Images. It's called Image Augmentation. so that the CNN model Doesn't get Overfitted onto our Training set.

In [2]:
# rescale = 1./255 - As we kno each pixel has values between 0 to 255, Hence we are applying Feature scaling on each Pixel
# by dividing it by 255 to make sure that the resultatnt value is between 0 to 255.
# Rest are the Geometrical transformations
train_datagen = ImageDataGenerator(rescale = 1./255,  
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

# Now we need to connect it to our training dataset
training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (64, 64),  # resizing the Images for faster training
                                                 batch_size = 32,         # no of images in each batch.
                                                 class_mode = 'binary')   # As we have a binary outcome we will choose 'binary'

Found 8000 images belonging to 2 classes.


### Preprocessing the Test set

In [3]:
# Now We will take the test dataset and we will apply only feature scaling and not the rest transformations.
test_datagen = ImageDataGenerator(rescale = 1./255)

# Now let's connect this dtagenerator Object to our test data.
test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 2000 images belonging to 2 classes.


## Part 2 - Building the CNN

In [4]:
# Initialising the CNN
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [5]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

### Step 2 - Pooling
* pool_size - Size of the overlapping matrix
* strides - Shifts
<img src='max pooling 03.png' width=500>

In [6]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [7]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [8]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [9]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Step 5 - Output Layer

In [10]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN
### Compiling the CNN

In [11]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [12]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 25)

Epoch 1/25
250/250 [==============================] - 123s 492ms/step - loss: 0.6881 - accuracy: 0.5419 - val_loss: 0.6921 - val_accuracy: 0.5635
Epoch 2/25
250/250 [==============================] - 57s 228ms/step - loss: 0.6307 - accuracy: 0.6496 - val_loss: 0.5846 - val_accuracy: 0.6930
Epoch 3/25
250/250 [==============================] - 55s 220ms/step - loss: 0.5738 - accuracy: 0.7060 - val_loss: 0.5386 - val_accuracy: 0.7330
Epoch 4/25
250/250 [==============================] - 60s 238ms/step - loss: 0.5369 - accuracy: 0.7311 - val_loss: 0.5160 - val_accuracy: 0.7505
Epoch 5/25
250/250 [==============================] - 58s 234ms/step - loss: 0.5084 - accuracy: 0.7560 - val_loss: 0.4875 - val_accuracy: 0.7605
Epoch 6/25
250/250 [==============================] - 64s 255ms/step - loss: 0.4845 - accuracy: 0.7650 - val_loss: 0.4846 - val_accuracy: 0.7695
Epoch 7/25
250/250 [==============================] - 66s 265ms/step - loss: 0.4691 - accuracy: 0.7768 - val_loss: 0.4578 - val_a

## Part 4 - Making a single prediction

In [22]:
import numpy as np
from keras.preprocessing import image

# Loading the Image and Resizing it as we have trained the model on 64 X 64 Images.  
test_image = image.load_img('dataset/single_prediction/cat_or_dog_2.jpg', target_size = (64, 64))

# As the Predict method axpects the data to be in an Array formt converting Img to an Array.
test_image = image.img_to_array(test_image)

# While training the Model we gave the images in the batch of 32 imges. that means the firts dimension of each image is the
# batch no. hence while predicting the result for this new single Image we need to add and specify the batch size.
# Hence adding a fake dimensuion to the image.
test_image = np.expand_dims(test_image, axis = 0) 

# predicting the result for a single Image.
result = cnn.predict(test_image)


training_set.class_indices
if result[0][0] == 1:
    prediction = 'dog'
else:
    prediction = 'cat'
    
print(prediction)

dog
